<a href="https://colab.research.google.com/github/patience275/llms/blob/main/chatbot_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers accelerate torch

In [2]:
from huggingface_hub import login
login()

In [3]:
import torch
from transformers import AutoModelForCausalLM,AutoTokenizer,TextStreamer
from accelerate import infer_auto_device_map,dispatch_model
model_name="TinyLlama/TinyLlama-1.1B-Chat-v1.0"
model=AutoModelForCausalLM.from_pretrained(model_name,torch_dtype=torch.float16)
tokenizer=AutoTokenizer.from_pretrained(model_name)
streamer=TextStreamer(tokenizer,skip_prompt=True,skip_specil_tokens=True)
device_map=infer_auto_device_map(model)
model=dispatch_model(model,device_map=device_map)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/608 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.20G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/551 [00:00<?, ?B/s]

In [4]:
chat_history = []
print('Your chatbot is ready! Type "e" to quit.\n')

while True:
    user_input = input('you: ')

    if user_input.lower() in ["e"]:
        print('chatbot: goodbye!')
        break

    # Add user message to chat history
    chat_history.append({'role': 'user', 'content': user_input})
    prompt = ''

    # Build the prompt from chat history
    for turn in chat_history:
        role = turn['role']
        content = turn['content']

        if role == 'user':
            prompt += f'<|user|>\n{content}\n'
        else:
            prompt += f'<|assistant|>\n{content}\n'

    prompt += '<|assistant|>\n'  # Signal that it's the assistant's turn

    # Tokenize the prompt
    inputs = tokenizer(prompt, return_tensors='pt', return_attention_mask=True).to(model.device)

    # Generate a response
    output = model.generate(
        **inputs,
        max_new_tokens=200,
        do_sample=True,
        temperature=0.7,
        streamer=streamer,
        pad_token_id=tokenizer.eos_token_id
    )

    # Decode the generated output
    output_text = tokenizer.decode(output[0], skip_special_tokens=True)

    # Extract assistant reply
    assistants_reply = output_text.split('<|assistant|>')[-1].strip()

    # Save and print assistant reply
    chat_history.append({'role': 'assistant', 'content': assistants_reply})
    print(f'\nchatbot: {assistants_reply}\n')


Your chatbot is ready! Type "e" to quit.

you: capital of kenya?
Yes, the capital city of Kenya is Nairobi.</s>

chatbot: Yes, the capital city of Kenya is Nairobi.

you: what is the capital of jamaica?
Yes, the capital city of Jamaica is Kingston.</s>

chatbot: Yes, the capital city of Jamaica is Kingston.

you: e
chatbot: goodbye!


In [6]:
# 1. Save the tokenizer (no changes needed)
tokenizer.save_pretrained("./saved_tinyllama")

# 2. Unwrap and save the full model
from accelerate import FullyShardedDataParallelPlugin

# Save the original model (on CPU)
model.cpu()
model.save_pretrained("./saved_tinyllama", safe_serialization=True)

In [7]:
from transformers import AutoModelForCausalLM, AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("./saved_tinyllama")
model = AutoModelForCausalLM.from_pretrained("./saved_tinyllama", torch_dtype=torch.float16)
